In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, optimizers, losses, Model
import networkx as nx
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import os

# Constants
STATE_SIZE = 10
NOISE_DIM = 10
ACTION_SIZE = 5
BATCH_SIZE = 32
EPISODES = 100

# GPT-2 model setup for text-based decision influence
class GPT2Transformer:
    def __init__(self):
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        self.model = GPT2LMHeadModel.from_pretrained("gpt2")
        self.tokenizer.pad_token = self.tokenizer.eos_token  # Ensure pad token is set

    def encode_text(self, text):
        inputs = self.tokenizer(text, return_tensors="tf")
        outputs = self.model(inputs)
        return outputs.last_hidden_state

    def generate_text(self, state):
        num_nodes = state[0]
        num_edges = state[1] if len(state) > 1 else 0
        state_description = f"Current state has {num_nodes} neurons and {num_edges} synapses."
        input_text = f"Based on the state: {state_description}, the optimal action is:"

        inputs = self.tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
        input_ids = inputs['input_ids'].to(self.model.device)
        attention_mask = inputs['attention_mask'].to(self.model.device)

        outputs = self.model.generate(input_ids, attention_mask=attention_mask, max_length=50)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

# Define GNN architecture for graph processing with Hebbian learning update
class GNNLayer(tf.keras.layers.Layer):
    def __init__(self, state_size):
        super(GNNLayer, self).__init__()
        self.state_size = state_size
        self.adjacency_matrix = tf.Variable(tf.eye(state_size), trainable=False)

    def call(self, inputs):
        inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
        if len(inputs.shape) == 1:
            inputs = tf.expand_dims(inputs, axis=0)
        x = tf.matmul(self.adjacency_matrix, tf.transpose(inputs))
        return tf.nn.relu(x)

    def hebbian_update(self, activations, learning_rate=0.01):
        delta_weights = learning_rate * tf.matmul(activations, activations, transpose_a=True)
        updated_adjacency_matrix = self.adjacency_matrix + delta_weights
        self.adjacency_matrix.assign(tf.clip_by_value(updated_adjacency_matrix, 0, 1))

# Define DDQN agent with curiosity and metacognition
class DDQNAgent:
    def __init__(self, state_size, action_size, learning_rate=0.001, gamma=0.99, curiosity_factor=0.01):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.curiosity_factor = curiosity_factor
        self.q_network = self._build_model()
        self.target_network = self._build_model()
        self.target_network.set_weights(self.q_network.get_weights())
        self.optimizer = optimizers.Adam(learning_rate=self.learning_rate)
        self.loss_function = losses.MeanSquaredError()

    def _build_model(self):
        return tf.keras.Sequential([
            layers.Dense(64, input_dim=self.state_size, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(self.action_size, activation='linear')
        ])
    
    def save_model(self, file_path):
        self.q_network.save(file_path)

    def load_model(self, file_path):
        self.q_network = tf.keras.models.load_model(file_path)
    def act(self, state, epsilon):
       
       if np.random.rand() <= epsilon:
          return np.random.randint(self.action_size)
    
    # Reshape state to match expected input shape
       state = np.reshape(state, (1, -1))  # -1 allows numpy to infer the correct dimension
       q_values = self.q_network.predict(state)
       return np.argmax(q_values[0])
    def train(self, state, action, reward, next_state, done):
        target = reward + (self.gamma * np.amax(self.target_network.predict(next_state)[0]) if not done else 0)
        target_f = self.q_network.predict(state)
        target_f[0][action] = target

        with tf.GradientTape() as tape:
            q_values = self.q_network(state)
            loss = self.loss_function(target_f, q_values)

        grads = tape.gradient(loss, self.q_network.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.q_network.trainable_variables))

    def update_target_network(self):
        self.target_network.set_weights(self.q_network.get_weights())

# GAN architecture to generate synthetic brain structures
class GAN:
    def __init__(self, state_size, noise_dim, learning_rate=0.001):
        self.state_size = state_size
        self.noise_dim = noise_dim
        self.generator = self._build_generator()
        self.discriminator = self._build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate), metrics=['accuracy'])
        self.gan_model = self._build_gan_model()

    def _build_generator(self):
        return tf.keras.Sequential([
            layers.Dense(16, input_dim=self.noise_dim, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(self.state_size, activation='tanh')
        ])

    def _build_discriminator(self):
        return tf.keras.Sequential([
            layers.Dense(32, input_dim=self.state_size, activation='relu'),
            layers.Dense(16, activation='relu'),
            layers.Dense(1, activation='sigmoid')
        ])

    def _build_gan_model(self):
        self.discriminator.trainable = False
        noise_input = layers.Input(shape=(self.noise_dim,))
        generated_state = self.generator(noise_input)
        validity = self.discriminator(generated_state)
        model = Model(noise_input, validity)
        model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam())
        return model

    def train(self, real_states, batch_size, epochs):
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, real_states.shape[0], batch_size)
            real_batch = real_states[idx]
            noise = np.random.normal(0, 1, (batch_size, self.noise_dim))
            generated_states = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_batch, valid)
            d_loss_fake = self.discriminator.train_on_batch(generated_states, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            g_loss = self.gan_model.train_on_batch(noise, valid)

    def generate(self, batch_size):
        noise = np.random.normal(0, 1, (batch_size, self.noise_dim))
        return self.generator.predict(noise)

# Brain environment that integrates GNN, DDQN, GAN, and GPT-2
class BrainEnvironment:
    def __init__(self, graph, gan, gnn, gpt2):
        self.graph = graph
        self.gan = gan
        self.gnn = gnn
        self.gpt2 = gpt2
        self.state_size = STATE_SIZE

    def add_neuron(self):
        new_node = len(self.graph.nodes)
        self.graph.add_node(new_node)
        self.gnn.update_adjacency_matrix(np.random.rand(self.state_size))

    def step(self, action):
        self.grow_brain_structure(action)
        return self.get_state()

    def get_state(self):
        state = update_state(self.graph)
        if len(state) < self.state_size:
            state = np.pad(state, (0, self.state_size - len(state)), 'constant')
        elif len(state) > self.state_size:
            state = state[:self.state_size]
        return state

    def grow_brain_structure(self, action):
        if action == "add_neuron":
            self.add_neuron()
        elif action == "add_synapse":
            self.add_synapse()

    def add_synapse(self):
        if len(self.graph.nodes) > 1:
            node1, node2 = np.random.choice(self.graph.nodes, 2, replace=False)
            if not self.graph.has_edge(node1, node2):
                self.graph.add_edge(node1, node2)

# Update state representation to include graph metrics
def update_state(graph):
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()
    synapses = num_edges  # Assuming each edge represents a synapse
    state = np.array([num_nodes, num_edges, synapses] + [0] * (STATE_SIZE - 3))  # Pad with zeros
    return state

# Reward calculation based on graph properties
def calculate_reward(graph):
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()
    if num_nodes == 0:
        return 0
    if num_edges == 0:
        return num_nodes * 0.1

    avg_clustering = nx.average_clustering(graph)
    return num_nodes * 0.1 + num_edges * 0.2 + avg_clustering * 0.5

def modify_graph(graph, action):
    if action == "add_node":
        graph.add_node(len(graph.nodes))
    elif action == "add_edge":
        if len(graph.nodes) > 1:
            node1, node2 = np.random.choice(graph.nodes, 2, replace=False)
            if not graph.has_edge(node1, node2):
                graph.add_edge(node1, node2)

    # Return the updated state after modifying the graph
    return update_state(graph)

# Training loop
def train(episodes):
    graph = nx.Graph()
    gpt2 = GPT2Transformer()
    gan = GAN(STATE_SIZE, NOISE_DIM)
    gnn = GNNLayer(STATE_SIZE)
    agent = DDQNAgent(STATE_SIZE, ACTION_SIZE)

    for episode in range(episodes):
        state = np.zeros((1, STATE_SIZE))  # Initial state
        done = False
        total_reward = 0
        
        while not done:
            action = agent.act(state, epsilon=0.1)
            next_state = modify_graph(graph, action)

            if next_state is None:
                print("Error: next_state is None.")
                break
            
            reward = calculate_reward(graph)
            total_reward += reward

            # Ensure state and next_state have the right shape
            state_reshaped = np.reshape(state, (1, STATE_SIZE))  # Add batch dimension
            next_state_reshaped = np.reshape(next_state, (1, STATE_SIZE))  # Add batch dimension

            agent.train(state_reshaped, action, reward, next_state_reshaped, done)
            state = next_state

            if done:
                print(f"Episode {episode}, Total Reward: {total_reward}")

        agent.update_target_network()
    
if __name__ == "__main__":
    train(EPISODES)


c:\Users\knigh\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step

c:\Users\knigh\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━

KeyboardInterrupt: 